In [347]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker  
from sklearn import linear_model
import statsmodels.api as sm

pd.set_option("display.precision", 8)

In [348]:
#we assume risk free rate remains stable at 0.13
#R_f = 0.13
#we can no longer assume

In [320]:
#load data
industry_portfolio_df = pd.read_excel('Industry_Portfolios.xlsx')
market_portfolio_df = pd.read_excel('Market_Portfolio.xlsx')
risk_factors_df = pd.read_excel('Risk_Factors.xlsx')


c:\Users\kilo\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
c:\Users\kilo\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
c:\Users\kilo\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [321]:
# use merge for join inner i guess
concated_df = industry_portfolio_df\
            .merge(market_portfolio_df, on ='Date', how='inner')
concated_df = concated_df\
            .merge(risk_factors_df, on ='Date', how='inner')
#concated_df

In [367]:
portfolio_cols = concated_df.columns[1:12]
portfolio_cols

Index(['NoDur', 'Durbl', 'Manuf', 'Enrgy', 'HiTec', 'Telcm', 'Shops', 'Hlth',
       'Utils', 'Other', 'Market'],
      dtype='object')

In [323]:
#concated_no_date_df= concated_df.drop(['Date'],axis =1)

In [324]:
excess_risk_df  = pd.DataFrame()
for x in portfolio_cols:
    excess_risk_df[f'{x}_excess']=concated_df[x] - concated_df['Rf']
#excess_risk_df

In [394]:
concated_df = pd.concat([concated_df, excess_risk_df], axis=1 )

In [392]:
sharpe = excess_risk_df.mean().to_numpy() / np.sqrt(excess_risk_df.var().to_numpy())

In [391]:
# regression for market model
market_model_result =[]
for portfolio in portfolio_cols:
    #print(portfolio)
    y = concated_df[portfolio].to_numpy()
    x = concated_df['Market'].to_numpy()
    market_model_result.append(np.polyfit(x,y,1)[::-1])

In [428]:
market_model_result_intercept = []
market_model_result_slope = []
for x in market_model_result:
    market_model_result_intercept.append(x[0])
    market_model_result_slope.append(x[1])
market_model_result_intercept =np.array(market_model_result_intercept)
market_model_result_slope = np.array(market_model_result_slope)

In [429]:
treynor = excess_risk_df.mean().to_numpy() / market_model_result_slope
treynor

array([1.18836509, 0.36764952, 0.75700828, 1.13823449, 0.56400462,
       0.83728453, 0.95473755, 0.97555499, 1.44942727, 0.29970223,
       0.62083333])

In [433]:
jensen_alpha = excess_risk_df.mean().to_numpy() - excess_risk_df['Market_excess'].mean() * market_model_result_slope 
jensen_alpha = ['{:f}'.format(item) for item in jensen_alpha]
jensen_alpha

['0.370398',
 '-0.417383',
 '0.159304',
 '0.501801',
 '-0.064385',
 '0.194964',
 '0.275969',
 '0.238740',
 '0.445855',
 '-0.387705',
 '-0.000000']

In [434]:
downside_df = excess_risk_df.where(excess_risk_df > 0, 0, inplace=False)
semivariance = (downside_df**2).mean()
#semivariance

In [435]:
sortino = excess_risk_df.mean().to_numpy()/ (np.sqrt(semivariance)).to_numpy()
sortino

array([0.29562588, 0.09506544, 0.2262164 , 0.23784943, 0.16394142,
       0.22925482, 0.24750601, 0.22018023, 0.29619862, 0.09672374,
       0.19867397])

In [436]:
# regression for three factor alpha
market_factor = concated_df['Market']
size_factor = concated_df['SMB']
value_factor = concated_df['HML']
x = pd.concat([market_factor, size_factor, value_factor] , axis =  1 )


In [437]:
excess_risk_df

,NoDur_excess,Durbl_excess,Manuf_excess,Enrgy_excess,HiTec_excess,Telcm_excess,Shops_excess,Hlth_excess,Utils_excess,Other_excess,Market_excess
0,-0.01,-1.14,-0.69,0.37,4.46,1.34,0.38,3.02,1.85,2.81,2.15
1,4.19,-0.13,1.89,4.63,-2.98,-0.58,6.03,0.83,2.01,2.10,1.40
2,-0.18,-1.24,-0.36,-0.22,-2.64,-2.16,0.20,-4.05,1.04,-0.72,-1.32
3,1.34,2.22,-0.25,2.44,-4.99,-0.56,-2.78,3.46,-3.63,-3.84,-1.83
4,-1.95,-1.70,1.55,0.33,4.79,-3.01,0.24,-0.48,1.22,1.80,1.17
...,...,...,...,...,...,...,...,...,...,...,...
115,-4.00,-1.00,-1.95,-1.83,-0.12,-3.28,-4.21,-3.49,-4.43,-3.66,-2.69
116,1.94,6.46,5.20,2.03,3.77,4.33,4.46,4.19,1.05,3.83,3.76
117,4.75,0.72,4.56,4.86,3.80,5.52,5.08,3.47,3.35,3.87,4.17
118,1.29,1.81,2.95,0.85,3.65,0.27,3.67,4.54,-1.81,4.74,3.12


In [438]:
three_factor_alpha = []
for port in excess_risk_df.columns:
    regr = linear_model.LinearRegression()
    y = excess_risk_df[port]
    regr.fit(x, y)
    three_factor_alpha.append(regr.intercept_)
three_factor_alpha = np.array(three_factor_alpha)
three_factor_alpha

array([ 0.29543682, -0.65976326,  0.00494171,  0.38767973, -0.21561205,
        0.0822595 ,  0.15829688,  0.16217743,  0.39198016, -0.54851183,
       -0.12884985])

In [441]:
result = pd.DataFrame([sharpe, sortino, treynor, jensen_alpha, three_factor_alpha]).T
result.rename(index=[''])

TypeError: 'list' object is not callable